In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy import stats

In [143]:
# https://stackoverflow.com/questions/21546739/load-data-from-txt-with-pandas
#https://stackoverflow.com/questions/40011531/in-pandas-when-using-read-csv-how-to-assign-a-nan-to-a-value-thats-not-the



# Read data, change missing values to 'NaN'

Xtrain_raw = pd.read_csv('Xtrain.txt', sep =" ", index_col=0, dtype='float64',
                    na_values=['?','NaN'])
Xtest_raw = pd.read_csv('Xtest.txt', sep =" ", index_col=0, dtype='float64',
                    na_values=['?','NaN'])

Ytrain_raw = pd.read_csv('Ytrain.txt', sep =",", index_col=0, dtype='float64',
                    na_values=['?','NaN'])

In [144]:
Ytrain_raw.shape

(153287, 1)

In [145]:
Xtrain_raw.shape

(153287, 75)

In [146]:
Xtest_raw.shape
#Xtest_raw.head

(50000, 75)

In [147]:
#https://stackoverflow.com/questions/51207491/function-to-replace-nan-values-in-a-dataframe-with-mean-of-the-related-column
#Replace NaN with column means in Xtrain
Xtrain_filled = Xtrain_raw.apply(lambda x: x.fillna(x.mean())) 
#print(Xtrain_filled.info())

In [148]:
#Do the same for other data-sets

Ytrain_filled = Ytrain_raw.apply(lambda x: x.fillna(x.mean())) 
Xtest_filled = Xtest_raw.apply(lambda x: x.fillna(x.mean())) 

In [149]:
#Concatenate the training and test sets and check to make sure that all observations from the dataset are there

Xmerged_raw = pd.concat([Xtrain_raw, Xtest_raw])
print('X_filled shape: ', Xtest_raw.shape)
#Xmerged_raw.info()

X_filled shape:  (50000, 75)


In [150]:
Xraw_sorted = Xmerged_raw.sort_index(axis=0)
#Xraw_sorted.info()

In [151]:
#Drop the last ten rows (all -9 in test set)
Xraw_clean=Xraw_sorted.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)
#print(Xraw_clean)

In [152]:
#save as a csv file for use in R

Xraw_clean.to_csv('/Users/Jamie/Xraw_clean.csv', index = True)



In [153]:
#Do the same for the component data sets

Xtrain_clean=Xtrain_filled.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)
Xtest_clean=Xtest_filled.drop(labels=[ '#H04', '#H05', '#H06', '#H07', '#H08', '#H09','#I01', '#I02', '#I03','#I04'], axis=1)

In [154]:
#Xfilled_sorted = Xraw_clean.apply(lambda x: x.fillna(x.mean()))
#Xfilled_sorted.info

In [155]:
#Merge training data sets
train_merged= Xtrain_clean.merge(Ytrain_filled, how='left', on='Id')
#print(train_merged)

In [156]:
#Remove the Id column and reset the index

train_merged_resetindex = train_merged.reset_index()
train_merged_noindex = train_merged_resetindex.drop(['Id'], axis=1)

In [157]:
#print(train_merged_noindex)

In [158]:
#Modeling 
#[some code based on code written for JAson Cao for project 2: train-test split and template for model fitting]
#https://github.com/MikeZ77/STAT-440/blob/master/nft/jason/NFT_regression-template.ipynb]



from sklearn.linear_model import LinearRegression, SGDRegressor,Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor

In [159]:

y = train_merged['Value'].copy()
X = train_merged.drop(['Value'], axis=1).copy()

In [160]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=32)

In [161]:
#Try Ridge Regression

In [162]:
# standardize for sample sets and Xtest
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
Xtest_clean_transformed= scaler.transform(Xtest_clean)

In [163]:
RidgeModel = Ridge(alpha=20).fit(x_train, y_train)

In [164]:
Ridge_predictions = RidgeModel.predict(x_test)

In [165]:
mean_squared_error(y_test, Ridge_predictions, squared=False)

9.64121745271793

In [166]:

ridge_g_search = GridSearchCV(estimator = Ridge(), param_grid = {'alpha': [1.0, 2.0, 3.0, 6.0, 10, 15, 20, 30, 50, 60]}, cv = 3, n_jobs = 1, verbose = 2, return_train_score=True)

In [167]:
ridge_g_search.fit(x_train, y_train);
print(ridge_g_search.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ..........................................alpha=1.0; total time=   0.1s
[CV] END ..........................................alpha=2.0; total time=   0.1s
[CV] END ..........................................alpha=2.0; total time=   0.1s
[CV] END ..........................................alpha=2.0; total time=   0.2s
[CV] END ..........................................alpha=3.0; total time=   0.1s
[CV] END ..........................................alpha=3.0; total time=   0.1s
[CV] END ..........................................alpha=3.0; total time=   0.3s
[CV] END ..........................................alpha=6.0; total time=   0.2s
[CV] END ..........................................alpha=6.0; total time=   0.3s
[CV] END .......................................

In [168]:
RidgeModel = Ridge(alpha=20).fit(x_train, y_train)

In [169]:
Ridge_predictions = RidgeModel.predict(x_test)

In [170]:
mean_squared_error(y_test, Ridge_predictions, squared=False)

9.64121745271793

In [171]:
#Try Stochastic Gradient Descent

In [172]:
SGDtest = SGDRegressor(loss = 'huber', alpha = 0.000001, epsilon = 0).fit(x_train, y_train)

In [173]:
SGD_test_predictions = SGDtest.predict(x_test)

In [174]:
mean_squared_error(y_test, SGD_test_predictions, squared=False)

42.34306606129916

In [175]:
param_grid = {'alpha':[0.1, 0.001,],
              'learning_rate':['adaptive'],
              'max_iter':[400, 600],
              'penalty':['l2','l1','elasticnet'],
              'loss':['huber', 'squared_epsilon_insensitive', 'squared_error'],
              'eta0': [0.01, 1, 10]}

In [176]:
sgd_ = SGDRegressor(random_state = 1)
g_search = GridSearchCV(estimator = sgd_, param_grid = param_grid, cv = 3, n_jobs = 1, verbose = 2, return_train_score=True)

In [177]:
#g_search.fit(x_train, y_train);
#print(g_search.best_params_)

In [178]:
sgdtest = SGDRegressor(alpha = 0.001, learning_rate= 'adaptive', loss = 'squared_epsilon_insensitive', max_iter= 400 ).fit(x_train, y_train)

In [179]:
SGD_test_predictions = sgdtest.predict(x_test)

In [180]:
mean_squared_error(y_test, SGD_test_predictions, squared=False)

9.641046549127175

In [181]:
#Try Adaboost

In [182]:
ABregr = AdaBoostRegressor(random_state=1, loss='square', n_estimators=80, learning_rate=0.01)
ABModel = ABregr.fit(x_train, y_train)

In [183]:
AB_test_predictions = ABModel.predict(x_test)

In [184]:
mean_squared_error(y_test, AB_test_predictions, squared=False)

10.08978372421733

In [185]:
param_grid = {'random_state':[1], 'n_estimators':[80],
              'learning_rate':[.01, 0.5, 1.0, 3.0],
              'loss':['linear', 'square', 'exponential']}

In [186]:
ab_ = AdaBoostRegressor(random_state = 1)
ABg_search = GridSearchCV(estimator = ab_, param_grid = param_grid, cv = 3, n_jobs = 1, verbose = 2, return_train_score=True)

In [187]:
#ABg_search.fit(x_train, y_train);
#print(ABg_search.best_params_)

In [188]:
ABregr = AdaBoostRegressor(random_state=1, loss='square', n_estimators=80, learning_rate=0.01)
ABModel = ABregr.fit(x_train, y_train)

In [189]:
AB_test_predictions = ABModel.predict(x_test)

In [190]:
mean_squared_error(y_test, AB_test_predictions, squared=False)

10.08978372421733

In [191]:
SGD_predictions = sgdtest.predict(Xtest_clean_transformed)

In [192]:
AB_predictions = ABModel.predict(Xtest_clean_transformed)

In [193]:
#Load the sample predictions
pred = pd.read_csv('pred.txt', sep =",", index_col=0, dtype= {'Value': np.float64})        

In [194]:
#print(pred)

In [195]:
#Now Generate predictions for SGD

sassa_pred_2 = pred.copy()
sassa_pred_2['prediction'] = SGD_predictions
sassa_pred_2.to_csv('/Users/Jamie/sassa_pred_2.csv', index = True)

In [196]:
#Now Generate predictions for AdaBoost

sassa_pred_3 = pred.copy()
sassa_pred_3['prediction'] = AB_predictions
sassa_pred_3.to_csv('/Users/Jamie/sassa_pred_3.csv', index = True)